In [2]:
OPENAI_API_KEY = ""
TRAVILY_API = "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"


In [2]:
!pip install -r requirements.txt

In [3]:
import os

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from tavily import TavilyClient


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
llama3_format_template = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

In [5]:
# query = "What is agent memory?" # Can answer
query = "What is Harry Potter?" # Can answer with Tavily
# query = "What is morning menu?" # Cannot answer

# Without Grpah

In [6]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

print(f"Question: {query}")

local_llm = "llama3"

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    ]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=200,
    chunk_overlap=50,
)
doc_splits = text_splitter.split_documents(docs_list)


embeddings_model = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    embedding=embeddings_model,
    collection_name="sol_day3",
    persist_directory="./chroma_db",
)
retriever = vectorstore.as_retriever()

# Common setting for chains
llm = ChatOllama(model=local_llm, format="json", temperature=0)
parser = JsonOutputParser()

coldstart = True
while 1:
    if coldstart:
        print("<<<<< Retrieval >>>>>")
        coldstart = False
        retrieved_docs = retriever.get_relevant_documents(query)
        candidate_context_list = [{"title": doc.metadata["title"], "content": doc.page_content, "url": doc.metadata["source"]} for doc in retrieved_docs]
    else:
        print("<<<<< Web search >>>>>")
        tavily = TavilyClient(api_key=TRAVILY_API)

        response = tavily.search(query=query, max_results=3)
        answer = response["answer"]
        candidate_context_list = [{"title": obj["title"], "content": obj["content"], "url": obj["url"]} for obj in response['results']]

    # Relevance check
    print("<<<<< Relevance check >>>>>")
    prompt = PromptTemplate(
        template=llama3_format_template.format(
            prompt="""Check the <query> is relevant to retrieved <context>. Answer with "yes" or "no" in json format(key: relevance).
            Here is the question: {query}
            Here is the context: {context}"""
        ),
        input_variables=["query", "context"],
    )
    relevant_chain = prompt | llm | parser
    valid_context_list = []
    for context in candidate_context_list:
        content = context["content"]
        is_relevant = relevant_chain.invoke({"query": query, "context": content})["relevance"]
        if is_relevant == "yes":
            valid_context_list.append(context)

    # Drop duplicate in context
    context_list = []
    for new_context in valid_context_list:
        if new_context["content"] not in [context["content"] for context in context_list]:
            context_list.append(new_context)
    if context_list:
        print("<<<<< Relevant documents found >>>>>")
        break

while 1:
    # Answer
    context = "\n".join([doc["content"] for doc in context_list])
    prompt = PromptTemplate(
        template=llama3_format_template.format(
            prompt="""Answer to the <query> with <context>. Answer in json format(key: answer).
            Here is the question: {query}
            Here is the context: {context}"""
        ),
        input_variables=["query", "context"],
    )
    question_chain = prompt | llm | parser
    answer = question_chain.invoke(
        {"query": query, "context": context}
        )["answer"]
    
    # Halucination check
    print("<<<<< Halucination check >>>>>")
    hallucination_prompt = PromptTemplate(
        template=llama3_format_template.format(
            prompt="""Check the <answer> has halllucination or not. Answer with "yes" or "no" in json format(key: hallucination).
            Here is the answer: {answer}"""
        ),
        input_variables=["answer"],
    )
    hallucination_chain = hallucination_prompt | llm | parser
    hallucination = hallucination_chain.invoke({"answer": answer})["hallucination"]

    if hallucination == "no":
        break    

# print
print("<<<<< Answering >>>>>")
print("="*50)
print(f"Answer: {answer}")
for idx, context in enumerate(context_list):
    print("-"*50)
    print(f"Context {idx+1}")
    print(f"Title: {context['title']}")
    print(f"Content: {context['content']}")
    print(f"Source: {context['url']}")
print("="*50)
print("<<<<< Finished >>>>>")


Question: What is Harry Potter?
<<<<< Retrieval >>>>>


/Users/sol.ty/Desktop/workspace/ai-advanced/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


<<<<< Relevance check >>>>>
<<<<< Web search >>>>>
<<<<< Relevance check >>>>>
<<<<< Relevant documents found >>>>>
<<<<< Halucination check >>>>>
<<<<< Answering >>>>>
Answer: Harry Potter is an orphan who is mistreated by his guardian aunt and uncle and their son, and is forced to battle Voldemort and his disciples multiple times.
--------------------------------------------------
Context 1
Title: Harry Potter | Character, Books, Movies, & Facts | Britannica
Content: Series summary
Harry Potter was first introduced in the novel Harry Potter and the Philosopher’s Stone (1997; also published as Harry Potter and the Sorcerer’s Stone), as an orphan who is mistreated by his guardian aunt and uncle and their son. Harry and those who side with him—including some of his teachers, several classmates, and other members of the wizarding community—are forced on multiple occasions to battle Voldemort and his disciples, including the Malfoy family. In the fourth volume, Harry Potter and the Goblet